# Patent Reduction

## Purpose

The purpose of this program is to identify the core similiarities between 42 patents and applications, and reduce their number to the top 3 or 4 based on ngrams and other factors.

## Work Plan

- Get full text of patents
- Analyze ngrams of pantents
- Identify embeddings 
- Conduct dimension reduction technique

In [2]:
import pandas as pd
import os

In [8]:
url ='./Patents/gp-search-20170622-225739.csv'

In [9]:
df = pd.read_csv(url, skiprows=1)

In [10]:
df.head()

,id,title,assignee,inventor/author,priority date,filing/creation date,publication date,grant date,result link
0,US7286245B2,Method and apparatus for determining the influ...,Carl Zeiss Smt Ag,"Ulrich Wegmann, Michael Hartl, Markus Mengel, ...",2002-07-29,2003-07-29,2007-10-23,2007-10-23,https://patents.google.com/patent/US7286245B2/en
1,US20080043236A1,Optical Property Measurement Apparatus and Opt...,Nikon Corporation,"Koji Kaise, Toru Fujii, Yasushi Mizuno",2004-08-09,2005-08-09,2008-02-21,NaN,https://patents.google.com/patent/US2008004323...
2,US7277182B2,Apparatus for polarization-specific examinatio...,Carl Zeiss Smt Ag,"Ulrich Wegmann, Markus Mengel",2003-07-05,2004-07-06,2007-10-02,2007-10-02,https://patents.google.com/patent/US7277182B2/en
3,US6788389B2,Production method of projection optical system,Nikon Corporation,"Youhei Fujishima, Hironori Ikezawa, Toshihiko ...",2001-07-10,2002-07-10,2004-09-07,2004-09-07,https://patents.google.com/patent/US6788389B2/en
4,US6870668B2,Method for evaluating image formation performa...,Nikon Corporation,Toshihiko Ozawa,2000-10-10,2001-10-09,2005-03-22,2005-03-22,https://patents.google.com/patent/US6870668B2/en


'https://patents.google.com/patent/US20080043236A1/en'

In [18]:
import spacy
from collections import Counter, defaultdict


In [14]:
nlp = spacy.load('en')
doc = nlp(u'This is a sentence. This is a golfing green')

In [15]:

# Get first token of the processed document
token = doc[0]
print(token)

# Print sentences (one sentence per line)
for sent in doc.sents:
    print(sent)

This
This is a sentence.
This is a golfing green


In [19]:
#article = read_file('data/article.txt')
article = 'A method and an apparatus for determining the influencing of the state of polarization of optical radiation by an optical system under test, wherein radiation with a defined entrance state of polarization is directed onto the optical system, the exit-side state of polarization is measured, and the influencing of the state of polarization is determined by the optical system with the aid of evaluation of the exit state of polarization with reference to the entrance state of polarization. An analyser arrangement which can be used for this purpose is also disclosed. The method and the apparatus are used, e.g., to determine the influencing of the state of polarization of optical radiation by an optical imaging system of prescribable aperture, the determination being performed in a pupil-resolved fashion.'
doc = nlp(article)

keywords = Counter()
for chunk in doc.noun_chunks:
    if nlp.vocab[chunk.lemma_].prob < - 8: # probablity value -8 is arbitrarily selected threshold
        keywords[chunk.lemma_] += 1

keywords.most_common(20)


[('polarization', 7),
 ('the influencing', 3),
 ('the state', 3),
 ('optical radiation', 2),
 ('the optical system', 2),
 ('an optical system', 1),
 ('test', 1),
 ('radiation', 1),
 ('a define entrance state', 1),
 ('the exit - side state', 1),
 ('the aid', 1),
 ('evaluation', 1),
 ('the exit state', 1),
 ('reference', 1),
 ('the entrance state', 1),
 ('an analyser arrangement', 1),
 ('this purpose', 1),
 ('the method', 1),
 ('the apparatus', 1),
 ('an optical imaging system', 1)]